In [1]:
import pandas as pd
import numpy as np
from itertools import product
import statsmodels.api as sm
import statsmodels.formula.api as smf
from tqdm import tqdm

# ARDL

In [2]:
df=pd.read_csv('data_other_cpi.csv')

In [3]:
df.head()

,date,CPI_chain,CPI_chaincloth,CPI_chainhouse,CPI_chainlive,CPI_chaincommu,CPI_chainedu,CPI_chainmed,CPI_chainothers,m_CPI_chaincloth,...,diesel,yk_famelec,yk_kitchen,before,during,after,year,month,season,covid
0,Jan-11,0.3,-0.5,0.4,0.4,0.5,0.4,0.3,NaN,NaN,...,NaN,NaN,NaN,0.86,0.0,0.0,2011,1,4,0.0
1,Feb-11,0.1,-0.8,0.3,0.2,0.1,0.3,0.2,NaN,NaN,...,0.021574,NaN,NaN,0.14,1.0,1.0,2011,2,4,0.0
2,Mar-11,0.2,0.5,0.5,0.3,0.0,-0.6,0.2,NaN,NaN,...,0.023875,NaN,NaN,0.00,0.0,0.0,2011,3,1,0.0
3,Apr-11,0.4,0.6,0.4,0.2,0.4,0.4,0.3,NaN,NaN,...,0.048289,NaN,NaN,0.00,0.0,0.0,2011,4,1,0.0
4,May-11,0.2,0.5,0.2,0.3,0.1,0.0,0.4,NaN,NaN,...,0.000993,NaN,NaN,0.00,0.0,0.0,2011,5,1,0.0


In [5]:
def clean_data(df,y,x,max_lag):

    df1=df.set_index('date')
    data=df1.loc[:,y+x].copy() # 挑选出自变量和因变量所在列
    def create_lag(data,y,x,max_lag):
        for lag in range(1,max_lag+1): # 滞后阶数lag=1---max_lag
            data[y[0]+'_'+str(lag)]=data[y].shift(-lag) # 制作因变量的滞后项
            for index in range(len(x)): # 同时，制作自变量对应滞后项
                data[x[index]+'_'+str(lag)]=data[x[index]].shift(-lag)
        return data # y,x1,x2,...,y_1,x_1,x_2,...
    data=create_lag(data,y,x,max_lag) # 因变量+自变量+自变量1阶+...+
    
    return data

In [11]:
def ardl(df,y,x,max_lag):
    
    df1=df.set_index('date')
    data=df1.loc[:,y+x].copy() # 挑选出自变量和因变量所在列
    def clean_data(data,y,x,max_lag):
        for lag in range(1,max_lag+1): # 滞后阶数lag=1---max_lag
            data[y[0]+'_'+str(lag)]=data[y].shift(-lag) # 制作因变量的滞后项
            for index in range(len(x)): # 同时，制作自变量对应滞后项
                data[x[index]+'_'+str(lag)]=data[x[index]].shift(-lag)
        return data # y,x1,x2,...,y_1,x_1,x_2,...
    data=clean_data(data,y,x,max_lag) # 因变量+自变量+自变量1阶+...+
    
    # dic{0:[y,[y,y_1],[y,y_1,y_2],[y,y_1,y_2,.....]],1:....}
    dic={}
    def clean_list(lis): # 将lis=[y,y_1,y_2,y_3.....]，变为[y,[y,y_1],[y,y_1,y_2],[y,y_1,y_2,.....]]
        for index in range(1,len(lis)): # 从list的第二个元素开始，后者总是包含前者
            lis[index]=lis[index-1]+'+'+lis[index] # lis[i]=lis[i]+lis[i-1]
        return lis
    for i in range(len(x)+1): # 因变量+自变量个数等=len(x)+1
        lis=[] #  当i=0时，对应因变量y
        for j in range(max_lag+1): # 变量本身+滞后项个数=max_lag+1
            # y,x1,x2,...,y_1,x_1,x_2,... 将变量及其滞后项挑出来
            lis.append(list(data.columns)[i+j*(len(x)+1)]) # 周期为因变量+自变量个数等=len(x)+1
        if i == 0: # 因变量y不包含变量本身
            del lis[0] 
            dic[i]=clean_list(lis) # 将lis合并到dic中
        else:
            dic[i]=clean_list(lis)
    
    # 下列代码利用递归自动实现函数动态参数
    # 下列代码等价于list(product(dic[0],dic[1],dic[2],dic[3],...,dic[max_lag]))
    def f(n):
        if n==0:
            return dic[n]
        else: # 普通的递归结果是(d,(c,(b,a))),我们要变成(a,b,c,d)
            return tuple_to_one_list(list(product(dic[n],f(n-1))))
    def tuple_to_one_list(Permutation):
        variable_name=[]
        for i in Permutation: # i=(b,a) or i=(c,[a,b]) or ....
            i=list(i) # i=[b,a] or i=[c,[a,b]] or ....
            i.reverse() # i=[a,b] or i=[[a,b],c] or ....
            lis=[]
            for j in i: # j=a,b or j=[a,b],c
                if type(j)==list: # if j=[a,b]
                    for t in j:
                        lis.append(t) # lis=[a,b]
                else:
                    lis.append(j) # lis=[a,b] or lis=[a,b,c]
            variable_name.append(lis) # variable_name=[[a,b]] or variable_name=[[a,b],[a,b,c]]
        return variable_name
    variable=f(len(x)) # 至此variable包含了在最大滞后阶数max_lag下，全部变量组合
    # 例如 variable[0]=['CPI_chaincloth_1', 'clothes', 'before', 'during', 'after', 'covid']
    
    # 制作可被写入smf.ols()函数中的变量字符串
    # 例如：'y~ x1+x2'
    variable_list=[]
    for v in variable:
        text='CPI_chaincloth~ '
        for i in v:
            text=text+'+'+i
        variable_list.append(text)
    
    # 以最小aic为标准，寻找ols结果里aic最小的变量组合
    aic=float("inf") # 设：aic=∞
    best_reg=0
    for variab in tqdm(variable_list): #所有的变量组合都在variable_list里
        reg = smf.ols(variab,data).fit() # variab是一个具体的变量组合
        if reg.aic<aic:
            aic=reg.aic
            best_reg=variab
    reg = smf.ols(best_reg,data).fit()
    print(best_reg,aic)
    #a=reg.summary()
    #b=reg.predict()
    #b1=b.to_frame()
    #b1.reset_index(inplace=True)
    return reg

In [5]:
y=['CPI_chaincloth']
x=['m_CPI_chaincloth','clothes','before','during','after','covid']

In [6]:
ardl(df,y,x,max_lag=4)

100%|██████████████████████████████████████████████████████████████| 62500/62500 [41:03<00:00, 25.37it/s]

CPI_chaincloth~ +CPI_chaincloth_1+CPI_chaincloth_2+m_CPI_chaincloth+m_CPI_chaincloth_1+clothes+before+before_1+before_2+before_3+during+during_1+during_2+during_3+during_4+after+after_1+after_2+after_3+after_4+covid -88.39712166475485


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         CPI_chaincloth   R-squared:                       0.945
Model:                            OLS   Adj. R-squared:                  0.927
Method:                 Least Squares   F-statistic:                     53.14
Date:                Sun, 05 Jul 2020   Prob (F-statistic):           1.60e-31
Time:                        11:05:11   Log-Likelihood:                 65.199
No. Observations:                  83   AIC:                            -88.40
Df Residuals:                      62   BIC:                            -37.60
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -0.0480      0.024     -2.014      0.048      -0.096      -0.000
CPI_chaincloth_1       0.2383      0.127      1.880      0.065      -0.015       0.492
CPI_chaincloth_2       0.0840      0.056      1.510      0.136      -0.027       0.195
m_CPI_chaincloth       0.9357      0.062     15.086      0.000       0.812       1.060
m_CPI_chaincloth_1    -0.2372      0.128     -1.858      0.068      -0.492       0.018
clothes               -1.4310      1.564     -0.915      0.364      -4.558       1.696
before                -0.2585      0.377     -0.686      0.495      -1.012       0.495
before_1               0.4308      0.441      0.977      0.332      -0.451       1.312
before_2               0.3423      0.421      0.813      0.419      -0.500       1.184
before_3               1.1857      0.364      3.254      0.002       0.457       1.914
during                -0.0637      0.177     -0.360      0.720      -0.417       0.290
during_1              -0.1162      0.208     -0.560      0.578      -0.532       0.299
during_2              -0.0327      0.209     -0.156      0.877      -0.451       0.386
during_3              -0.3827      0.187     -2.049      0.045      -0.756      -0.009
during_4               0.1725      0.112      1.542      0.128      -0.051       0.396
after                  0.2816      0.101      2.802      0.007       0.081       0.483
after_1                0.3238      0.386      0.838      0.405      -0.448       1.096
after_2               -0.3862      0.443     -0.873      0.386      -1.271       0.498
after_3               -0.3764      0.419     -0.898      0.373      -1.214       0.461
after_4               -1.1698      0.350     -3.345      0.001      -1.869      -0.471
covid                  0.0499      0.176      0.284      0.777      -0.302       0.402
==============================================================================
Omnibus:                        0.838   Durbin-Watson:                   1.990
Prob(Omnibus):                  0.658   Jarque-Bera (JB):                0.639
Skew:                          -0.215   Prob(JB):                        0.726
Kurtosis:                       2.994   Cond. No.                         128.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [143]:
ardl(df,y,x,max_lag=5)

100%|██████████████████████████████████████████████████████████| 233280/233280 [2:59:47<00:00, 21.62it/s]


CPI_chaincloth~ +CPI_chaincloth_1+m_CPI_chaincloth+m_CPI_chaincloth_1+m_CPI_chaincloth_2+m_CPI_chaincloth_3+m_CPI_chaincloth_4+m_CPI_chaincloth_5+clothes+before+during+after+after_1+after_2+after_3+covid+covid_1+covid_2+covid_3 -93.21791239543936


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         CPI_chaincloth   R-squared:                       0.947
Model:                            OLS   Adj. R-squared:                  0.931
Method:                 Least Squares   F-statistic:                     61.99
Date:                Sat, 04 Jul 2020   Prob (F-statistic):           3.81e-33
Time:                        21:17:24   Log-Likelihood:                 65.609
No. Observations:                  82   AIC:                            -93.22
Df Residuals:                      63   BIC:                            -47.49
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              0.1397      0.067      2.078      0.042       0.005       0.274
CPI_chaincloth_1       0.1570      0.119      1.318      0.192      -0.081       0.395
m_CPI_chaincloth       0.8729      0.076     11.452      0.000       0.721       1.025
m_CPI_chaincloth_1    -0.3233      0.137     -2.358      0.021      -0.597      -0.049
m_CPI_chaincloth_2    -0.0371      0.079     -0.472      0.639      -0.194       0.120
m_CPI_chaincloth_3    -0.0934      0.070     -1.340      0.185      -0.233       0.046
m_CPI_chaincloth_4    -0.0070      0.065     -0.107      0.915      -0.138       0.124
m_CPI_chaincloth_5    -0.2632      0.077     -3.437      0.001      -0.416      -0.110
clothes               -2.7521      1.503     -1.831      0.072      -5.756       0.252
before                -0.3767      0.296     -1.275      0.207      -0.967       0.214
during                 0.0039      0.158      0.025      0.980      -0.312       0.320
after                  0.0472      0.117      0.404      0.688      -0.187       0.281
after_1                0.1771      0.271      0.654      0.515      -0.364       0.718
after_2               -0.1851      0.103     -1.804      0.076      -0.390       0.020
after_3               -0.2858      0.090     -3.160      0.002      -0.467      -0.105
covid                 32.8561     16.763      1.960      0.054      -0.641      66.353
covid_1              -11.5832      5.861     -1.976      0.052     -23.295       0.128
covid_2                4.0854      1.990      2.053      0.044       0.108       8.062
covid_3               -1.1662      0.501     -2.327      0.023      -2.168      -0.165
==============================================================================
Omnibus:                        0.492   Durbin-Watson:                   1.750
Prob(Omnibus):                  0.782   Jarque-Bera (JB):                0.148
Skew:                           0.068   Prob(JB):                        0.929
Kurtosis:                       3.157   Cond. No.                     1.52e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.52e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""